<a href="https://colab.research.google.com/github/MarioCid90/PRUEBA2/blob/main/AWS/Proyectochatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importamos librerías
import random
import json
import pickle
import numpy as np

import nltk #librería para trabajar con palabras
from nltk.stem import WordNetLemmatizer # coge el array de cada paralabra
from keras.models import Sequential #modelo secuancial
from keras.layers import Dense, Activation, Dropout #capas de la red neuronal
from keras.optimizers import SGD #optimizador para la red neuronal.



In [2]:
lemmatizer = WordNetLemmatizer() # llamamos al algoritmo para convertir las palabras a 1 y 0

In [3]:
intents = json.loads(open('intents.json').read()) #cargamos el archivo json

In [4]:
#Estos modelos permiten dividir el texto en oraciones de manera efectiva.
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
#Creamos listas
words = []
classes = []
documents = []
ignore_letters = ['?','!','¿','.', ','] #No nos influyen estos simbolos

In [6]:

for intent in intents['intenciones']: #acceder a todos los patrones del json
  for pattern in intent['texto']: #patrones que en json llamamos texto
    word_list = nltk.word_tokenize(pattern) #facilita convertir todo a 1 y 0
    words.extend(word_list) #La lista de palabras que hemos creado van a pasar por esta función
    documents.append((word_list, intent['texto'])) #relaciona las palabras con el identificador
    if intent['texto'] not in classes: #Si no hemos añadido el identificador
      classes.append(intent['texto']) #que me las relacione igual

In [7]:
#LAs palabras que hemos separado de los patrones, las devolvemos a la palabra raiz
#para que no haya confusiones a la hora de entrenarlo
words= [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
#ordenamos las palabras y las convertimos a un SET
words = sorted(set(words))

In [8]:
#Guardar las palabras

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [9]:

training = [] #lista de entrenamiento
output_empty = [0] * len(classes) #crea tantos 0 como patrones tenemos

for document in documents: #
  bag = [] #lista con un monton de variables
  word_patters = document[0] #todas las palabras que tenemos en la word_list
  word_patters = [lemmatizer.lemmatize(word.lower()) for word in word_patters] #
  for word in words: #para cada palabra en esta lista añadimos 1 y la palabra está en word_patters
    bag.append(1) if word in word_patters else bag.append(0) #1 si la palabra pertenece al patrón
#0 si la palabra no pertenece al patrón
  output_row = list(output_empty) #variable que es la lista con todos los 1 y 0
  output_row[classes.index(document[1])] = 1 #cogemos el indice de todas las clases = 1
  training.append([bag, output_row]) #añadir a la lista final las palabras y la lista


In [10]:
#dividimos todos los datos en 2 listas, una con los 0 y otra con los 1
train_X=list(map(lambda x: x[0], training))
train_Y=list(map(lambda x: x[1], training))


In [11]:
#Creamos capas de la red neuronal

model= Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation='softmax'))

In [12]:
#vamos a definir el optimizador SGD

sgd = SGD(learning_rate=0.001,  momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



In [13]:
#Entrenamos el modelo
train_proces =model.fit(np.array(train_X), np.array(train_Y), epochs=4, batch_size=5, verbose=1)

Epoch 1/4
83/83 [==============================] - 1s 2ms/step - loss: 3.6700 - accuracy: 0.0316
Epoch 2/4
83/83 [==============================] - 0s 3ms/step - loss: 3.6544 - accuracy: 0.0364
Epoch 3/4
83/83 [==============================] - 0s 4ms/step - loss: 3.6397 - accuracy: 0.0583
Epoch 4/4
83/83 [==============================] - 0s 3ms/step - loss: 3.6186 - accuracy: 0.0874


In [ ]:
#Guardamos en un archivo todo el proceso de entrenamiento

model.save('chatbot_model.h5', train_proces)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#importación de keras para cargar la data que acabamos de guardar
from keras.models import load_model



In [ ]:
lemmatizer = WordNetLemmatizer() #llamamos al algoritmo nuevamente
intents = json.loads(open('intents.json').read()) #cargamos el archivo json
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))



In [ ]:
#llamamos al modelo entrenado
model = load_model('chatbot_model.h5')

In [ ]:
#Esta funcion devuelve lista de palabras bien organizadas
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [ ]:
#Si una de las palabras está dentro de las palabras definidas, nos la transforma en 1
#Si no a 0
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

In [ ]:
#predice a la clase que pertenece la oración que escribimos.
#devuelve el indice con más probabilidad de paracerse a la clase.
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  max_index = np.where(res == np.max(res))[0][0]
  return classes[max_index]

In [ ]:
#Función para que nos devuelva una respuesta
def get_response(intents_list, intents_json):
  list_of_intents = intents_json['intenciones']
  for i in list_of_intents:
    if i['texto'] == intents_list:
      result = random.choice(i['respuestas'])
      break
  return result



In [ ]:
while True:
    message = input(" ")

    # Agrega una condición para salir del bucle
    if message.lower() == 'salir':
        print("¡Hasta luego!")
        break  # Sale del bucle while

    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

 Hola
1/1 [==============================] - 0s 122ms/step
¡Qué bueno verte de nuevo!
 cursos
1/1 [==============================] - 0s 22ms/step
Nuestra universidad ofrece tecnología de la información, ingeniería informática, ingeniería mecánica, ingeniería química, ingeniería civil e ingeniería externa
 horarios
1/1 [==============================] - 0s 24ms/step
¡La universidad está abierta de 8 am a 5 pm de lunes a sábado!
 instalaciones
1/1 [==============================] - 0s 24ms/step
El departamento de ingeniería de nuestra universidad ofrece un laboratorio de aire acondicionado completo con conexión a Internet, aula inteligente, auditorio, biblioteca y comedor
 contacto
1/1 [==============================] - 0s 21ms/step
Puedes contactar al: NÚMERO
 como se llama el director
1/1 [==============================] - 0s 25ms/step
XYZ es el director de la universidad y si necesita ayuda, llame primero a su sucursal. Eso es más apropiado
 salir
¡Hasta luego!
